#### Initializing Libraries

In [1]:
import torch
import numpy as np
import cv2 as cv 
from time import time, sleep

#### Model download

In [2]:
best_model_path = 'G:\level 3 term 2\LAB EEE 318\Project\Script\one_best_640_93_5_2_yoloV5s.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=best_model_path, force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\abc/.cache\torch\hub\master.zip
YOLOv5  2023-2-26 Python-3.10.0 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Class Names, Device, Capture index

In [3]:
classes = model.names  #dictionary 
print(classes)
device = 'cuda' if torch.cuda.is_available() else 'cpu'  #string
print("Using Device: ", device) 
capture_index = 1

{0: 'metal', 1: 'paper', 2: 'plastic'}
Using Device:  cpu


Function Definition

In [4]:
# funtion for finding labels and cord. basically labels are needed. 
def score_frame(frame):
     model.to(device)
     frame = [frame]  # making it a list 
     results = model(frame)
     labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
     return labels, cord

In [5]:
# Fucntion for ectracting the class label 
def class_to_label(x):
    return classes[int(x)]

In [6]:
# Function for finding label and plotting
def plot_boxes(results, frame):
        labels, cord = results
        n = len(labels)
        exact_label = "" 
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.3:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv.putText(frame, class_to_label(labels[i]), (x1, y1), cv.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)
                exact_label = class_to_label(labels[i]) 
        return frame, exact_label 

### Camera opening Function Definition

In [7]:
def capture_trash_type():
    cap = cv.VideoCapture(0)  
    # assert cap.isOpened() 

    trash = []
    while True:
            sleep(0.5) # 0.5 sec
            ret, frame = cap.read()
            # assert ret
                
            frame = cv.resize(frame, (640,640))
                
            start_time = time()
            results = score_frame(frame)
            frame, exact_label = plot_boxes(results, frame) 
            print(exact_label)
                
            end_time = time()
            fps = 1/np.round(end_time - start_time, 2)
            #print(f"Frames Per Second : {fps}")
                
            cv.putText(np.array(frame), f'FPS: {int(fps)}', (20,70), cv.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
                
            cv.imshow('YOLOv5 Detection', frame)
            
            if exact_label: 
                trash.append(exact_label) 
                if len(list(set(trash))) > 1:
                    trash = []
                elif cv.waitKey(2) and len(trash) == 2 and len(list(set(trash))) == 1:
                    print('ok')
                    print(trash[0])
                    break  
            # if cv.waitKey(5) & 0xFF == ord('q'):
            #     cv.destroyAllWindows()
            #     break
    cv.destroyAllWindows()       
    # cap.release() 
    return trash[-1] 

In [ ]:
label = capture_trash_type()

### Arduino handling

In [19]:
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()
serialInst = serial.Serial()
portsList = []

for onePort in ports:
    portsList.append(str(onePort))
    print(str(onePort))

portVar = "COM5"
serialInst.baudrate = 9600
serialInst.port = portVar
serialInst.open() 

COM5 - USB-SERIAL CH340 (COM5)


In [24]:
command = capture_trash_type()
serialInst.write(command.encode('utf-8'))

paper
paper
ok
paper


5